In [1]:
# example of a super learner model for binary classification
from numpy import hstack
from numpy import vstack
from numpy import asarray
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
#create a list of base models
def get_models():
  models = []
  models.append(SVC(kernel='rbf',probability=True,random_state=0))
  models.append(XGBClassifier(max_depth=9, eta = 0.4, gamma = 3, n_rounds = 100))
  return models

In [3]:
# collect out of fold predictions form k-fold cross validation
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	# define split of data
	kfold = KFold(n_splits=10, shuffle=True)
	# enumerate splits
	for train_ix, test_ix in kfold.split(X):
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			# store columns
			fold_yhats.append(yhat)
		# store fold yhats as columns
		meta_X.append(hstack(fold_yhats))
	return vstack(meta_X), asarray(meta_y)

In [4]:
# fit all base models on the training dataset
def fit_base_models(X, y, models):
	for model in models:
		model.fit(X, y)

In [5]:
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model

In [6]:
# make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X)
		meta_X.append(yhat)
	meta_X = hstack(meta_X)
	# predict
	return meta_model.predict_proba(meta_X)

In [7]:
import io
import pandas as pd
df = pd.read_csv('X_trainData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [8]:
df

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,AMK
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,1,1,1,0,0,1,1,0,1
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,1,1,0,...,0,1,0,1,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
X = df.iloc[:,:-1].values
X.shape

(1360, 219)

In [10]:
y = df.iloc[:,-1].values
y.shape

(1360,)

In [11]:
# get models
models = get_models()
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
print('Meta ', meta_X.shape, meta_y.shape)

[13:41:07] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:41:09] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:41:12] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:41:14] WARNI

In [12]:
# fit base models
fit_base_models(X, y, models)

[13:41:32] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [13]:
# fit the meta model
meta_model = fit_meta_model(meta_X, meta_y)

In [14]:
X_test = pd.read_csv('final_X_testData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [15]:
import numpy as np

In [16]:
X_test

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,AMK
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
338,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,?
339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?


In [17]:
X_val = X_test.iloc[:,:-1].values
X_val.shape

(341, 219)

In [18]:
yhat = super_learner_predictions(X_val, models, meta_model)


In [19]:
yhat

array([[0.02928526, 0.97071474],
       [0.03468729, 0.96531271],
       [0.02814323, 0.97185677],
       [0.02859271, 0.97140729],
       [0.02839361, 0.97160639],
       [0.02814323, 0.97185677],
       [0.02943315, 0.97056685],
       [0.0309889 , 0.9690111 ],
       [0.02827225, 0.97172775],
       [0.02970116, 0.97029884],
       [0.02872403, 0.97127597],
       [0.02951254, 0.97048746],
       [0.02850582, 0.97149418],
       [0.02956088, 0.97043912],
       [0.02970728, 0.97029272],
       [0.02947099, 0.97052901],
       [0.02826406, 0.97173594],
       [0.02855967, 0.97144033],
       [0.02850297, 0.97149703],
       [0.02807301, 0.97192699],
       [0.02819549, 0.97180451],
       [0.02859271, 0.97140729],
       [0.02885818, 0.97114182],
       [0.02753102, 0.97246898],
       [0.02803837, 0.97196163],
       [0.02768739, 0.97231261],
       [0.02844379, 0.97155621],
       [0.02762792, 0.97237208],
       [0.02829429, 0.97170571],
       [0.03030025, 0.96969975],
       [0.

In [20]:
submission_data = pd.read_csv('Y_testData_1_nolabels_AMK.csv')

In [21]:
ids=submission_data.iloc[:,:1].values

In [22]:
result=list()

for i in range(len(ids)):
    result.append([ ids[i,0], yhat[i,1]])

result.insert(0, ['ID','AMK'])
result

[['ID', 'AMK'],
 [1, 0.9707147368082948],
 [2, 0.9653127089857995],
 [3, 0.9718567674520264],
 [4, 0.9714072912145582],
 [5, 0.9716063911630481],
 [6, 0.9718567674520264],
 [7, 0.9705668482300529],
 [8, 0.9690110995921339],
 [9, 0.9717277500938575],
 [10, 0.9702988395976693],
 [11, 0.9712759717529822],
 [12, 0.9704874638632232],
 [13, 0.9714941765157235],
 [14, 0.9704391179127989],
 [15, 0.970292719258567],
 [16, 0.9705290125423333],
 [17, 0.9717359411166064],
 [18, 0.9714403305730396],
 [19, 0.9714970259134234],
 [20, 0.9719269859405929],
 [21, 0.9718045135073635],
 [22, 0.9714072912145582],
 [23, 0.9711418244076034],
 [24, 0.9724689827076091],
 [25, 0.9719616304137206],
 [26, 0.9723126149374866],
 [27, 0.9715562107531234],
 [28, 0.9723720750010555],
 [29, 0.971705711668038],
 [30, 0.969699754123658],
 [31, 0.9707763324473676],
 [32, 0.9690773967353342],
 [33, 0.9705142412720656],
 [34, 0.9697027481249527],
 [35, 0.0950719399121889],
 [36, 0.9728295045070373],
 [37, 0.9705668482300529

In [23]:
output = pd.DataFrame(result)
output.to_csv('AMK_results.csv',header = False, index = False)